In [1]:
import glob
import os
from random import shuffle
from shutil import copyfile
import numpy as np
import json
import nibabel as nb

In [2]:
def subfiles(folder, join = True, prefix = None, suffix = None, sort = True):
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def save_json(obj, file: str, indent: int = 4, sort_keys: bool = True) -> None:
    with open(file, 'w') as f:
        json.dump(obj, f, sort_keys=sort_keys, indent=indent)

In [3]:
img = glob.glob('/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_*/full_nv_crossmoda_*_hrT2.nii.gz') + \
    glob.glob('/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/source_training/crossmoda_*/full_nv_crossmoda_*_ceT1.nii.gz')
lab = [[
    img_.replace('nv', 'c'+str(i)+'nv')
    for i in range(1,7)] for img_ in img]

In [4]:
print(img[0])
print(lab[0])

/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_nv_crossmoda_125_hrT2.nii.gz
['/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_c1nv_crossmoda_125_hrT2.nii.gz', '/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_c2nv_crossmoda_125_hrT2.nii.gz', '/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_c3nv_crossmoda_125_hrT2.nii.gz', '/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_c4nv_crossmoda_125_hrT2.nii.gz', '/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_c5nv_crossmoda_125_hrT2.nii.gz', '/Users/liamchalcroft/Desktop/MRES/Data/crossmoda/training/target_training/crossmoda_125/full_c6nv_crossmoda_125_hrT2.nii.gz']


In [5]:
im_ = nb.load(img[0])

In [6]:
im_.get_fdata().shape

(313, 377, 105)

In [7]:
odir = '/Users/liamchalcroft/Desktop/MRES/ssunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task003_CROSSMODA_PRETRAIN'
name = 'CROSSMODA_PRETRAIN'

In [8]:
os.makedirs(os.path.join(odir,'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(odir,'labelsTr'), exist_ok=True)
os.makedirs(os.path.join(odir,'imagesTs'), exist_ok=True)

In [9]:
for i, (img_, lab_) in enumerate(zip(img, lab)):
    img_load = nb.load(img_)
    nb.save(img_load, os.path.join(odir,'imagesTr',name+'_%04d_0000.nii.gz' % (i+1)))

    lab_load = [nb.load(label) for label in lab_]
    lab_data = np.stack([label.get_fdata() for label in lab_load])
    lab_data = lab_data[[5,0,1,2,3,4]]
    # lab_data = np.argmax(lab_data, axis=0)
    lab_data = nb.Nifti1Image(lab_data, lab_load[0].affine, lab_load[0].header)
    nb.save(lab_data, os.path.join(odir,'labelsTr',name+'_%04d.nii.gz' % (i+1)))        

In [10]:
def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-12] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques

In [11]:
train_identifiers = get_identifiers_from_splitted_files(os.path.join(odir,'imagesTr'))

json_dict = {}
json_dict['name'] = name
json_dict['description'] = 'Vestibular Schwanoma Segmentation'
json_dict['tensorImageSize'] = "4D"
json_dict['reference'] = "BLANK"
json_dict['licence'] = 'BLANK'
json_dict['release'] = 'BLANK'
json_dict['modality'] = {'0': 'T1/T2'}
json_dict['labels'] = {'0': 'Background', '1': 'WM', '2': 'GM', '3': 'CSF', '4': 'Non-brain', '5': 'Skull'}

json_dict['numTraining'] = len(train_identifiers)
json_dict['numTest'] = 0
json_dict['training'] = [
    {'image': os.path.join(odir,"imagesTr/%s.nii.gz" % i), 
    "label": os.path.join(odir,"labelsTr/%s.nii.gz" % i),
    } for i
    in
    train_identifiers]
json_dict['test'] = []

save_json(json_dict, os.path.join(odir,'dataset.json'))

In [12]:
json_dict['training'][0]

{'image': '/Users/liamchalcroft/Desktop/MRES/ssunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task003_CROSSMODA_PRETRAIN/imagesTr/CROSSMODA_PRETRAIN_0001.nii.gz',
 'label': '/Users/liamchalcroft/Desktop/MRES/ssunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task003_CROSSMODA_PRETRAIN/labelsTr/CROSSMODA_PRETRAIN_0001.nii.gz'}